In [68]:
import qdrant_client
import sentence_transformers
import langchain
import pandas

print("All imports OK ✅")


All imports OK ✅


In [69]:
!pip -q install pandas python-dotenv qdrant-client sentence-transformers
!pip -q install langchain langchain-openai


In [70]:
import os
from dotenv import load_dotenv
from qdrant_client import QdrantClient

load_dotenv()  # loads .env from project root (same level as backend/, data/, notebooks/)

QDRANT_URL = os.getenv("QDRANT_URL")
QDRANT_API_KEY = os.getenv("QDRANT_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")  

assert QDRANT_URL, "Missing QDRANT_URL in .env"
assert QDRANT_API_KEY, "Missing QDRANT_API_KEY in .env"

client = QdrantClient(url=QDRANT_URL, api_key=QDRANT_API_KEY)

print("✅ Connected to Qdrant:", QDRANT_URL)
print("Collections:", client.get_collections())


✅ Connected to Qdrant: https://db948a35-ef44-4b88-ad47-a65eb335dcbb.us-east4-0.gcp.cloud.qdrant.io
Collections: collections=[CollectionDescription(name='biocup_report_chunks_minilm384'), CollectionDescription(name='biocup_chunks')]


In [71]:
import pandas as pd
from pathlib import Path

CSV_PATH = ROOT / "data" / "processed" / "biocup_chunks.csv"
chunk_df = pd.read_csv(CSV_PATH)
print(chunk_df.columns)

print("Rows:", len(df))
print("Columns:", list(df.columns))


Index(['chunk_id', 'case_id', 'primary_site', 'tcga_type', 'patient_id',
       'section', 'original_section', 'chunk_index', 'sub_index', 'chunk_text',
       'has_tnm', 'has_size', 'has_ihc', 'has_lymph', 'has_margins',
       'has_tumor_size_cue', 'is_admin_noise'],
      dtype='str')
Rows: 420
Columns: ['case_id', 'primary_site', 'tcga_type', 'patient_id', 'patient_filename', 'report_text']


In [73]:

TEXT_COL = "report_text"
assert TEXT_COL in df.columns, "report_text column not found!"
df[TEXT_COL] = df[TEXT_COL].astype(str)

In [74]:
from sentence_transformers import SentenceTransformer

embed_model = SentenceTransformer("all-MiniLM-L6-v2")
vectors = embed_model.encode(chunk_df[TEXT_COL].tolist(), show_progress_bar=True).tolist()
VECTOR_SIZE = len(vectors[0])

print("✅ Embedded:", len(vectors))


KeyError: 'report_text'

In [13]:
from qdrant_client.http import models as qm

VECTOR_SIZE = 384

if not client.collection_exists(COLLECTION):
    client.create_collection(
        collection_name=COLLECTION,
        vectors_config=qm.VectorParams(size=VECTOR_SIZE, distance=qm.Distance.COSINE),
    )
    print("✅ Created collection:", COLLECTION)
else:
    print("ℹ️ Collection already exists:", COLLECTION)


ℹ️ Collection already exists: biocup_chunks


In [62]:
from sentence_transformers import SentenceTransformer
def upsert_in_batches(points, batch_size=50):
    for start in range(0, len(points), batch_size):
        batch = points[start:start + batch_size]
        client.upsert(collection_name=COLLECTION, points=batch)
        print(f"✅ upserted {start + len(batch)}/{len(points)}")

upsert_in_batches(points, batch_size=50)

TEXT_COL = "chunk_text"
embed_model = SentenceTransformer("all-MiniLM-L6-v2")

texts = df[TEXT_COL].astype(str).tolist()
vectors = embed_model.encode(texts, show_progress_bar=True).tolist()

points = []
for i, row in df.reset_index(drop=True).iterrows():
    payload = row.to_dict()
    points.append(qm.PointStruct(id=i, vector=vectors[i], payload=payload))

client.upsert(collection_name=COLLECTION, points=points)
print("✅ Uploaded:", len(points), "chunks to", COLLECTION)


✅ upserted 50/5089
✅ upserted 100/5089
✅ upserted 150/5089
✅ upserted 200/5089
✅ upserted 250/5089
✅ upserted 300/5089
✅ upserted 350/5089
✅ upserted 400/5089
✅ upserted 450/5089
✅ upserted 500/5089
✅ upserted 550/5089
✅ upserted 600/5089
✅ upserted 650/5089
✅ upserted 700/5089
✅ upserted 750/5089
✅ upserted 800/5089
✅ upserted 850/5089
✅ upserted 900/5089
✅ upserted 950/5089
✅ upserted 1000/5089
✅ upserted 1050/5089
✅ upserted 1100/5089
✅ upserted 1150/5089
✅ upserted 1200/5089
✅ upserted 1250/5089
✅ upserted 1300/5089
✅ upserted 1350/5089
✅ upserted 1400/5089
✅ upserted 1450/5089
✅ upserted 1500/5089
✅ upserted 1550/5089
✅ upserted 1600/5089
✅ upserted 1650/5089
✅ upserted 1700/5089
✅ upserted 1750/5089
✅ upserted 1800/5089
✅ upserted 1850/5089
✅ upserted 1900/5089
✅ upserted 1950/5089
✅ upserted 2000/5089
✅ upserted 2050/5089
✅ upserted 2100/5089
✅ upserted 2150/5089
✅ upserted 2200/5089
✅ upserted 2250/5089
✅ upserted 2300/5089
✅ upserted 2350/5089
✅ upserted 2400/5089
✅ upserted 2

KeyError: 'chunk_text'

In [65]:
def extract_points(resp):
    # Case 1: resp is a tuple (points, something)
    if isinstance(resp, tuple):
        return resp[0]
    # Case 2: resp is a QueryResponse-like object with .points
    if hasattr(resp, "points"):
        return resp.points
    # Case 3: resp is already a list of points
    return resp

query = "lung cancer case with no lymph node involvement"
qvec = embed_model.encode(query).tolist()

resp = client.query_points(
    collection_name=COLLECTION,
    query=qvec,
    limit=5
)

points = extract_points(resp)

for pt in points:
    p = pt.payload
    print("score:", round(pt.score, 3), "| case_id:", p.get("case_id"), "| section:", p.get("section"))
    print(str(p.get(TEXT_COL))[:220], "...\n")


score: 0.692 | case_id: BIOCUP_00047 | section: LYMPH_NODES
[case_id=BIOCUP_00047 | site=lung | type=LUAD | section=LYMPH_NODES] B. EMPHYSEMATOUS CHANGE WITH FOCAL ORGANIZING PNEUMONIA. C. MULTIPLE SMALL CARCINOID TUMORLETS. PART 2: LUNG, INFERIOR LEFT LOWER LOBE NODULE, WEDGE RE ...

score: 0.663 | case_id: BIOCUP_00021 | section: LYMPH_NODES
[case_id=BIOCUP_00021 | site=lung | type=LUSC | section=LYMPH_NODES] The lung adjacent to the tumor shows interstitial follicular pneumonia and early acute supporative pneumonia. The pleura is thick and hemorrahgic and a ...

score: 0.658 | case_id: BIOCUP_00041 | section: MICRO
[case_id=BIOCUP_00041 | site=lung | type=LUAD | section=MICRO] . Microscopic. Description. DIAGNOSIS Details. Comments. Laterality. LUNG TISSUE CHECKLIST. Left-upper. Specimen type: Lobectomy. Tumor site: Lung. Tumor siz ...

score: 0.649 | case_id: BIOCUP_00007 | section: GROSS
[case_id=BIOCUP_00007 | site=lung | type=LUSC | section=GROSS] There are n 0 satellite lesions

In [46]:
import os
from pathlib import Path
from dotenv import load_dotenv

# 1) Find repo root (folder that contains "notebooks")
HERE = Path.cwd()
ROOT = HERE if (HERE / "notebooks").exists() else HERE.parent

# 2) Load .env explicitly from root (no guessing)
dotenv_path = ROOT / ".env"
print("📌 cwd:", HERE)
print("📌 using .env at:", dotenv_path, "| exists:", dotenv_path.exists())

load_dotenv(dotenv_path=dotenv_path, override=True)

# 3) Verify
key = os.getenv("OPENAI_API_KEY")
print("OPENAI_API_KEY loaded?", bool(key))
if key:
    print("OPENAI_API_KEY preview:", key[:7] + "..." + key[-4:])


📌 cwd: /Users/mac/Documents/GitHub/BIOCUP1/notebooks
📌 using .env at: /Users/mac/Documents/GitHub/BIOCUP1/.env | exists: True
OPENAI_API_KEY loaded? True
OPENAI_API_KEY preview: sk-proj...aaMA


In [50]:
def extract_points(resp):
    # qdrant-client can return different shapes
    if isinstance(resp, tuple):
        return resp[0]
    if hasattr(resp, "points"):
        return resp.points
    return resp

def search(query: str, k: int = 5, query_filter=None):
    qvec = embed_model.encode(query).tolist()
    resp = client.query_points(
        collection_name=COLLECTION,
        query=qvec,
        limit=k,
        query_filter=query_filter,
        with_payload=True
    )
    return extract_points(resp)


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
import os

load_dotenv()

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are a clinical assistant for BioCUP.\n"
     "Rules:\n"
     "- Use ONLY the provided context.\n"
     "- If context is insufficient, say what is missing.\n"
     "- Cite sources as (case_id, chunk_index).\n"
     "- Do not invent medical facts.\n"),
    ("human",
     "Question:\n{question}\n\nContext:\n{context}\n\n"
     "Answer with bullet points + a short conclusion.")
])

chain = prompt | llm | StrOutputParser()

def format_context(points, max_chars=3500):
    out, total = [], 0
    for pt in points:
        p = pt.payload
        block = (
            f"(case_id={p.get('case_id')}, chunk_index={p.get('chunk_index')}, score={pt.score:.3f})\n"
            f"{p.get('chunk_text','')}"
        )
        out.append(block)
        total += len(block)
        if total > max_chars:
            break
    return "\n\n".join(out)

def rag(question: str, k: int = 6, query_filter=None):
    points = search(question, k=k, query_filter=query_filter)
    context = format_context(points)
    answer = chain.invoke({"question": question, "context": context})
    return answer, points

answer, pts = rag("Summarize lymph node status mentioned in similar lung cancer reports.", k=6)
print(answer)


- **Case BIOCUP_00030**: Three lymph nodes from the right superior hilar were negative for tumor; matted lymph node from the right upper node was also negative; right posterior superior hilar lymph node was negative for tumor (N0) (case_id=BIOCUP_00030, chunk_index=0).
  
- **Case BIOCUP_00055**: Eighteen lymph nodes from level VII were negative for malignancy; one lymph node from level X was negative; one lymph node from level XI was negative; one lymph node from level XII was negative (N0) (case_id=BIOCUP_00055, chunk_index=0).

- **Case BIOCUP_00039**: A lymph node biopsy from level 9 (left) was received, but no malignancy status was provided (case_id=BIOCUP_00039, chunk_index=16).

- **Case BIOCUP_00007**: Out of two peribronchial lymph nodes, one was positive for metastasis; six hilar lymph nodes were negative (0/6); several other lymph nodes were reported as benign (case_id=BIOCUP_00007, chunk_index=1).

- **Case BIOCUP_00016**: Four lymph nodes were examined, and none were posit